In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization, LSTM, GRU
from sklearn.utils import resample
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [2]:
# Load the dataset
df = pd.read_csv('statlog.csv', header = None)

df.columns = ['age', 'sex', 'cp', 'trestbps', 'chol',
              'fbs', 'restecg', 'thalach', 'exang', 
              'oldpeak', 'slope', 'ca', 'thal', 'target']

df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,70,1,4,130,322,0,2,109,0,2.4,2,3,3,2
1,67,0,3,115,564,0,2,160,0,1.6,2,0,7,1
2,57,1,2,124,261,0,0,141,0,0.3,1,0,7,2
3,64,1,4,128,263,0,0,105,1,0.2,2,1,7,1
4,74,0,2,120,269,0,2,121,1,0.2,1,1,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,52,1,3,172,199,1,0,162,0,0.5,1,0,7,1
266,44,1,2,120,263,0,0,173,0,0.0,1,0,7,1
267,56,0,2,140,294,0,2,153,0,1.3,2,0,3,1
268,57,1,4,140,192,0,0,148,0,0.4,2,0,6,1


In [3]:
### 1 = male, 0 = female
df.isnull().sum()

df['target'] = df.target.map({0: 0, 1: 0, 2: 1, 3: 1, 4: 1})
df['sex'] = df.sex.map({0: 'female', 1: 'male'})
df['thal'] = df.thal.fillna(df.thal.mean())
df['ca'] = df.ca.fillna(df.ca.mean())
df['sex'] = df.sex.map({'female': 0, 'male': 1})

In [4]:
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,70,1,4,130,322,0,2,109,0,2.4,2,3,3,1
1,67,0,3,115,564,0,2,160,0,1.6,2,0,7,0
2,57,1,2,124,261,0,0,141,0,0.3,1,0,7,1
3,64,1,4,128,263,0,0,105,1,0.2,2,1,7,0
4,74,0,2,120,269,0,2,121,1,0.2,1,1,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,52,1,3,172,199,1,0,162,0,0.5,1,0,7,0
266,44,1,2,120,263,0,0,173,0,0.0,1,0,7,0
267,56,0,2,140,294,0,2,153,0,1.3,2,0,3,0
268,57,1,4,140,192,0,0,148,0,0.4,2,0,6,0


In [5]:
# Separate features and target variable
X = df.drop('target', axis=1)
y = df['target']


In [6]:

# 1. Load and preprocess the data
# Assuming you have your data in the X and y variables

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)



In [7]:
input_shape = (X_train.shape[1], 1)
input_shape_cnn = (X_train.shape[1], 1)

In [8]:
# 4. Hyperparameter tuning
learning_rate = 0.001
batch_size = 50
epochs = 100



# 5. Learning rate scheduling
lr_scheduler = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=learning_rate, decay_steps=10000, decay_rate=0.9
)
optimizer = keras.optimizers.Adam(learning_rate=lr_scheduler)



# 6. Regularization techniques
callback_early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True
)
callback_checkpoint = keras.callbacks.ModelCheckpoint(
    'model_checkpoint.h5', save_best_only=True
)



In [9]:
def build_cnn_model(input_shape):
    model = Sequential()
    model.add(Conv1D(filters=11, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.25))
    model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))
    model.add(Dense(1, activation='sigmoid'))  # Output layer
    
    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model


In [10]:
# Train the individual models
cnn_model = build_cnn_model(input_shape)
cnn_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test),
             callbacks=[callback_early_stopping, callback_checkpoint])


Epoch 1/100
4/4 [==============================] - 2s 171ms/step - loss: 0.7576 - accuracy: 0.5370 - val_loss: 0.6667 - val_accuracy: 0.6944
Epoch 2/100
4/4 [==============================] - 0s 44ms/step - loss: 0.5157 - accuracy: 0.7716 - val_loss: 0.6529 - val_accuracy: 0.6852
Epoch 3/100
4/4 [==============================] - 0s 44ms/step - loss: 0.4919 - accuracy: 0.7963 - val_loss: 0.6481 - val_accuracy: 0.6944
Epoch 4/100
4/4 [==============================] - 0s 40ms/step - loss: 0.4770 - accuracy: 0.7593 - val_loss: 0.6464 - val_accuracy: 0.7037
Epoch 5/100
4/4 [==============================] - 0s 40ms/step - loss: 0.4149 - accuracy: 0.8272 - val_loss: 0.6424 - val_accuracy: 0.7037
Epoch 6/100
4/4 [==============================] - 0s 54ms/step - loss: 0.3250 - accuracy: 0.8519 - val_loss: 0.6405 - val_accuracy: 0.6944
Epoch 7/100
4/4 [==============================] - 0s 53ms/step - loss: 0.3601 - accuracy: 0.8519 - val_loss: 0.6401 - val_accuracy: 0.6759
Epoch 8/100
4/4 [==

4/4 [==============================] - 0s 21ms/step - loss: 0.0588 - accuracy: 0.9815 - val_loss: 0.5664 - val_accuracy: 0.6852
Epoch 60/100
4/4 [==============================] - 0s 24ms/step - loss: 0.0973 - accuracy: 0.9630 - val_loss: 0.5752 - val_accuracy: 0.7222
Epoch 61/100
4/4 [==============================] - 0s 23ms/step - loss: 0.0855 - accuracy: 0.9691 - val_loss: 0.5779 - val_accuracy: 0.6944
Epoch 62/100
4/4 [==============================] - 0s 20ms/step - loss: 0.0691 - accuracy: 0.9815 - val_loss: 0.5822 - val_accuracy: 0.6944
Epoch 63/100
4/4 [==============================] - 0s 23ms/step - loss: 0.1353 - accuracy: 0.9506 - val_loss: 0.5860 - val_accuracy: 0.6667
Epoch 64/100
4/4 [==============================] - 0s 20ms/step - loss: 0.1316 - accuracy: 0.9321 - val_loss: 0.5991 - val_accuracy: 0.6852
Epoch 65/100
4/4 [==============================] - 0s 21ms/step - loss: 0.0740 - accuracy: 0.9753 - val_loss: 0.6095 - val_accuracy: 0.7037
Epoch 66/100
4/4 [========

In [11]:
# 8. Evaluate the model
test_loss, test_accuracy = cnn_model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')


4/4 [==============================] - 0s 6ms/step - loss: 0.5631 - accuracy: 0.7222
Test Loss: 0.5631
Test Accuracy: 0.7222


In [12]:
# Make predictions
y_pred = cnn_model.predict(X_test)
y_pred_classes = np.round(y_pred)


# Calculate confusion matrix and classification report
from sklearn.metrics import confusion_matrix, classification_report


conf_matrix = confusion_matrix(y_test, y_pred_classes)
class_report = classification_report(y_test, y_pred_classes)

print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)


4/4 [==============================] - 0s 676us/step
Confusion Matrix:
[[52 11]
 [19 26]]

Classification Report:
              precision    recall  f1-score   support

           0       0.73      0.83      0.78        63
           1       0.70      0.58      0.63        45

    accuracy                           0.72       108
   macro avg       0.72      0.70      0.71       108
weighted avg       0.72      0.72      0.72       108



In [13]:
# 4. Hyperparameter tuning
learning_rate = 0.001
batch_size = 50
epochs = 100



# 5. Learning rate scheduling
lr_scheduler = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=learning_rate, decay_steps=10000, decay_rate=0.9
)
optimizer = keras.optimizers.Adam(learning_rate=lr_scheduler)



# 6. Regularization techniques
callback_early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True
)
callback_checkpoint = keras.callbacks.ModelCheckpoint(
    'model_checkpoint.h5', save_best_only=True
)



In [14]:
def build_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(11, return_sequences=True, input_shape=input_shape))
    model.add(LSTM(32, return_sequences=True))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(LSTM(64, return_sequences=True))
    model.add(LSTM(32))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Output layer
    
    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model


In [15]:
lstm_model = build_lstm_model(input_shape)
lstm_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test),
              callbacks=[callback_early_stopping, callback_checkpoint])


Epoch 1/100
4/4 [==============================] - 14s 1s/step - loss: 0.6830 - accuracy: 0.6296 - val_loss: 0.6864 - val_accuracy: 0.7222
Epoch 2/100
4/4 [==============================] - 0s 91ms/step - loss: 0.6268 - accuracy: 0.7469 - val_loss: 0.6742 - val_accuracy: 0.7222
Epoch 3/100
4/4 [==============================] - 0s 87ms/step - loss: 0.5774 - accuracy: 0.7346 - val_loss: 0.6526 - val_accuracy: 0.7315
Epoch 4/100
4/4 [==============================] - 0s 84ms/step - loss: 0.5415 - accuracy: 0.7284 - val_loss: 0.6275 - val_accuracy: 0.7315
Epoch 5/100
4/4 [==============================] - 0s 87ms/step - loss: 0.5154 - accuracy: 0.7531 - val_loss: 0.6198 - val_accuracy: 0.6759
Epoch 6/100
4/4 [==============================] - 0s 50ms/step - loss: 0.5367 - accuracy: 0.7654 - val_loss: 0.6204 - val_accuracy: 0.6574
Epoch 7/100
4/4 [==============================] - 0s 54ms/step - loss: 0.5236 - accuracy: 0.7407 - val_loss: 0.6255 - val_accuracy: 0.6481
Epoch 8/100
4/4 [====

In [16]:
# 8. Evaluate the model
test_loss, test_accuracy = lstm_model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')


4/4 [==============================] - 0s 13ms/step - loss: 0.6198 - accuracy: 0.6759
Test Loss: 0.6198
Test Accuracy: 0.6759


In [17]:
# Make predictions
y_pred = lstm_model.predict(X_test)
y_pred_classes = np.round(y_pred)


# Calculate confusion matrix and classification report
from sklearn.metrics import confusion_matrix, classification_report


conf_matrix = confusion_matrix(y_test, y_pred_classes)
class_report = classification_report(y_test, y_pred_classes)

print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)


4/4 [==============================] - 3s 47ms/step
Confusion Matrix:
[[57  6]
 [29 16]]

Classification Report:
              precision    recall  f1-score   support

           0       0.66      0.90      0.77        63
           1       0.73      0.36      0.48        45

    accuracy                           0.68       108
   macro avg       0.70      0.63      0.62       108
weighted avg       0.69      0.68      0.65       108



In [18]:
# 4. Hyperparameter tuning
learning_rate = 0.001
batch_size = 50
epochs = 100



# 5. Learning rate scheduling
lr_scheduler = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=learning_rate, decay_steps=10000, decay_rate=0.9
)
optimizer = keras.optimizers.Adam(learning_rate=lr_scheduler)



# 6. Regularization techniques
callback_early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True
)
callback_checkpoint = keras.callbacks.ModelCheckpoint(
    'model_checkpoint.h5', save_best_only=True
)



In [19]:
def build_gru_model(input_shape):
    model = Sequential()
    model.add(GRU(11, return_sequences=True, input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(GRU(32, return_sequences=True))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(GRU(64, return_sequences=True))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(GRU(128, return_sequences=True))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(GRU(128))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Output layer
    
    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model


In [20]:
gru_model = build_gru_model(input_shape)
gru_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test),
             callbacks=[callback_early_stopping, callback_checkpoint])


Epoch 1/100
4/4 [==============================] - 21s 1s/step - loss: 0.7260 - accuracy: 0.5494 - val_loss: 0.6911 - val_accuracy: 0.6481
Epoch 2/100
4/4 [==============================] - 1s 144ms/step - loss: 0.4663 - accuracy: 0.7716 - val_loss: 0.6901 - val_accuracy: 0.6296
Epoch 3/100
4/4 [==============================] - 1s 144ms/step - loss: 0.3668 - accuracy: 0.8519 - val_loss: 0.6889 - val_accuracy: 0.6204
Epoch 4/100
4/4 [==============================] - 1s 142ms/step - loss: 0.3515 - accuracy: 0.8642 - val_loss: 0.6877 - val_accuracy: 0.6574
Epoch 5/100
4/4 [==============================] - 1s 148ms/step - loss: 0.4098 - accuracy: 0.8210 - val_loss: 0.6872 - val_accuracy: 0.6759
Epoch 6/100
4/4 [==============================] - 0s 87ms/step - loss: 0.3481 - accuracy: 0.8827 - val_loss: 0.6877 - val_accuracy: 0.6667
Epoch 7/100
4/4 [==============================] - 0s 90ms/step - loss: 0.3118 - accuracy: 0.8519 - val_loss: 0.6880 - val_accuracy: 0.6944
Epoch 8/100
4/4 [

In [21]:
# 8. Evaluate the model
test_loss, test_accuracy = gru_model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')


4/4 [==============================] - 0s 17ms/step - loss: 0.6872 - accuracy: 0.6759
Test Loss: 0.6872
Test Accuracy: 0.6759


In [22]:
# Make predictions
y_pred = gru_model.predict(X_test)
y_pred_classes = np.round(y_pred)


# Calculate confusion matrix and classification report
from sklearn.metrics import confusion_matrix, classification_report


conf_matrix = confusion_matrix(y_test, y_pred_classes)
class_report = classification_report(y_test, y_pred_classes)

print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)


4/4 [==============================] - 3s 17ms/step
Confusion Matrix:
[[58  5]
 [30 15]]

Classification Report:
              precision    recall  f1-score   support

           0       0.66      0.92      0.77        63
           1       0.75      0.33      0.46        45

    accuracy                           0.68       108
   macro avg       0.70      0.63      0.61       108
weighted avg       0.70      0.68      0.64       108



In [23]:
# Make predictions using individual models
cnn_predictions = (cnn_model.predict(X_test) > 0.5).astype("int32")
lstm_predictions = (lstm_model.predict(X_test) > 0.5).astype("int32")
gru_predictions = (gru_model.predict(X_test) > 0.5).astype("int32")



4/4 [==============================] - 0s 13ms/step


In [24]:
# Combine predictions using a majority voting ensemble
ensemble_predictions = np.mean([cnn_predictions, lstm_predictions, gru_predictions], axis=0) >= 0.5
ensemble_predictions = ensemble_predictions.astype("int32")

In [25]:
# Calculate accuracy of the ensemble
ensemble_accuracy = accuracy_score(y_test, ensemble_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")

Ensemble Accuracy: 0.6944


In [26]:
# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, ensemble_predictions)

print("Confusion Matrix:")
print(conf_matrix)

# Print classification report
class_report = classification_report(y_test, ensemble_predictions)
print("Classification Report:")
print(class_report)

# Evaluate ensemble accuracy
ensemble_accuracy = accuracy_score(y_test, ensemble_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")


Confusion Matrix:
[[57  6]
 [27 18]]
Classification Report:
              precision    recall  f1-score   support

           0       0.68      0.90      0.78        63
           1       0.75      0.40      0.52        45

    accuracy                           0.69       108
   macro avg       0.71      0.65      0.65       108
weighted avg       0.71      0.69      0.67       108

Ensemble Accuracy: 0.6944


In [27]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.utils import resample
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from sklearn.model_selection import cross_val_score

In [28]:
# List of base models

lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train, y_train)

svm_l_model = SVC(kernel="linear")
svm_l_model.fit(X_train, y_train)

svm_p_model = SVC(kernel="poly")
svm_p_model.fit(X_train, y_train)

svm_r_model = SVC(kernel="rbf")
svm_r_model.fit(X_train, y_train)

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)


knn_model = KNeighborsClassifier()
knn_model.fit(X_train, y_train)

gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
gb_model.fit(X_train, y_train)

ab_model = AdaBoostClassifier(n_estimators=100, random_state=42)
ab_model.fit(X_train, y_train)

xgb_model = XGBClassifier(n_estimators=100, random_state=42)
xgb_model.fit(X_train, y_train)

gpc_model = GaussianProcessClassifier(random_state=42)
gpc_model.fit(X_train, y_train)

GaussianProcessClassifier(random_state=42)

In [29]:
lr_predictions = lr_model.predict(X_test)
svm_l_predictions = svm_l_model.predict(X_test)
svm_p_predictions = svm_p_model.predict(X_test)
svm_r_predictions = svm_r_model.predict(X_test)
rf_predictions = rf_model.predict(X_test)
knn_predictions = knn_model.predict(X_test)
gb_predictions = gb_model.predict(X_test)
ab_predictions = ab_model.predict(X_test)
xgb_predictions = xgb_model.predict(X_test)
gpc_predictions = gpc_model.predict(X_test)


In [30]:
# Combine predictions using ensemble (Voting Classifier)
ensemble_predictions = np.column_stack((cnn_predictions, lstm_predictions, gru_predictions, lr_predictions, 
                                        svm_l_predictions, svm_p_predictions, svm_r_predictions, 
                                        rf_predictions, knn_predictions, gb_predictions, 
                                        ab_predictions, xgb_predictions, gpc_predictions))


In [31]:
# Perform majority voting to get the final prediction
final_predictions = np.mean(ensemble_predictions, axis=1) >= 0.5
final_predictions = final_predictions.astype("int32")

In [32]:
# Calculate accuracy of the ensemble
ensemble_accuracy = accuracy_score(y_test, final_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")

Ensemble Accuracy: 0.7778


In [33]:
# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, final_predictions)

print("Confusion Matrix:")
print(conf_matrix)

# Print classification report
class_report = classification_report(y_test, final_predictions)
print("Classification Report:")
print(class_report)

# Evaluate ensemble accuracy
ensemble_accuracy = accuracy_score(y_test, final_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")

Confusion Matrix:
[[57  6]
 [18 27]]
Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.90      0.83        63
           1       0.82      0.60      0.69        45

    accuracy                           0.78       108
   macro avg       0.79      0.75      0.76       108
weighted avg       0.78      0.78      0.77       108

Ensemble Accuracy: 0.7778


In [34]:
# Combine predictions using ensemble (Voting Classifier)
ensemble_predictions = np.column_stack((cnn_predictions, lstm_predictions, gru_predictions, svm_r_predictions, 
                                        rf_predictions, knn_predictions, gb_predictions, 
                                        xgb_predictions, gpc_predictions))


In [35]:
# Perform majority voting to get the final prediction
final_predictions = np.mean(ensemble_predictions, axis=1) >= 0.5
final_predictions = final_predictions.astype("int32")

# Calculate accuracy of the ensemble
ensemble_accuracy = accuracy_score(y_test, final_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, final_predictions)

print("Confusion Matrix:")
print(conf_matrix)

# Print classification report
class_report = classification_report(y_test, final_predictions)
print("Classification Report:")
print(class_report)

# Evaluate ensemble accuracy
ensemble_accuracy = accuracy_score(y_test, final_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")

Ensemble Accuracy: 0.7685
Confusion Matrix:
[[56  7]
 [18 27]]
Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.89      0.82        63
           1       0.79      0.60      0.68        45

    accuracy                           0.77       108
   macro avg       0.78      0.74      0.75       108
weighted avg       0.77      0.77      0.76       108

Ensemble Accuracy: 0.7685
